# Building Multi-Agent Investment Firms with Strands SDK

## Introduction

Welcome to **Challenge 2** of the AWS re:Invent Jam! In this challenge, you'll build a sophisticated multi-agent investment analysis system using the Strands Agents SDK. This system mirrors the collaborative dynamics of real-world investment firms, where different specialists work together to make informed trading decisions.

### What You'll Build

You'll create an AI-powered investment research team consisting of:
- **Bull Researcher**: An optimistic analyst focused on growth opportunities and positive market factors
- **Bear Researcher**: A cautious analyst specializing in risk identification and market threats
- **Research Manager**: A senior decision-maker who synthesizes insights from both perspectives

### The Challenge Scenario

Imagine you're building the AI backbone for a modern investment firm. Your system needs to:
1. **Analyze Market Data**: Process financial information and news to understand market conditions
2. **Generate Diverse Perspectives**: Create both bullish and bearish investment cases
3. **Facilitate Structured Debates**: Allow agents to challenge and refine each other's arguments
4. **Synthesize Final Decisions**: Combine multiple viewpoints into actionable investment recommendations

### Technical Learning Objectives

By completing this challenge, you will:
- **Master Agent Design**: Learn to create specialized AI agents with distinct roles and expertise
- **Implement Multi-Agent Coordination**: Use the Strands Swarm framework for structured agent interactions
- **Build Collaborative AI Systems**: Demonstrate how multiple AI agents can work together effectively
- **Apply AI to Finance**: Create practical applications for investment analysis and decision-making

Let's begin building your multi-agent investment analysis system!


## Task 2 - Multi-Agent Investment Analysis System

### Objective
- **Primary Goal**: Master the fundamentals of Strands Agent API and build a sophisticated multi-agent investment analysis system
- **Specific Tasks**: 
  - Create Bull Researcher and Bear Researcher agents with distinct analytical perspectives
  - Implement collaborative dialogue and debate mechanisms between agents
  - Generate comprehensive market analysis reports and investment recommendations
  - Learn to utilize the Swarm multi-agent coordination framework effectively

### System Architecture
This system simulates the operational model of real-world investment firms, featuring these core components:
1. **Bull Researcher**: Focuses on identifying investment opportunities and positive market factors
2. **Bear Researcher**: Specializes in risk identification and negative market indicators  
3. **Research Manager**: Coordinates debates and synthesizes final investment decisions
4. **Multi-Agent Dialogue System**: Manages interactions and collaboration between agents

### Learning Outcomes
By completing this challenge, you will:
- Understand how to design and implement specialized AI agents with distinct roles
- Learn to orchestrate complex multi-agent conversations and debates
- Master the Strands SDK for building production-ready agent systems
- Gain insights into collaborative AI decision-making processes

## 1. Install Required Dependencies

### Installation Overview
This section installs all necessary packages for building our multi-agent investment analysis system. The dependencies are organized into several categories:

In [1]:
%pip install -r requirements.txt

  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached pydantic-2.11.7-py3-none-any.whl.metadata (67 kB)
  Using cached pydantic_core-2.33.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.8 kB)
  Using cached typing_inspection-0.4.1-py3-none-any.whl.metadata (2.6 kB)
  Using cached httpcore-1.0.9-py3-none-any.whl.metadata (21 kB)
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 949.2/949.2 kB 45.7 MB/s  0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 635.6/635.6 kB 44.9 MB/s  0:00:00
Using cached pydantic-2.11.7-py3-none-any.whl (444 kB)
Using cached pydantic_core-2.33.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (2.0

## 2. Prerequisites code  
### 2.1 Import Utils functions

In [1]:
# Import essential libraries for our multi-agent system
import sys
import os
from pathlib import Path
from default_config import DEFAULT_CONFIG  # Configuration settings for the system
import logging  # For debugging and monitoring agent interactions
from dotenv import load_dotenv  # For loading environment variables (API keys, etc.)

## 2. System Configuration and Setup

### 2.1 Import Core Libraries
First, we import the essential libraries needed for our multi-agent system:
- **System utilities**: For path management and configuration
- **Configuration management**: Default settings and environment variables
- **Logging**: For debugging and monitoring agent interactions

In [2]:
# Define the Large Language Model for our agents
# Amazon Nova Pro is optimized for complex reasoning and financial analysis
NOVA_RPO_MODEL_ID = "amazon.nova-pro-v1:0"

### 2.2 Configure Model Selection
We define the Large Language Model (LLM) that will power our agents. Amazon Nova Pro is a state-of-the-art model optimized for complex reasoning tasks, making it ideal for financial analysis and multi-agent debates.

### 2.3 Initialize LLM Model Factory

The `get_model()` function serves as our model factory, creating configured Amazon Bedrock model instances for our agents. Key features:

**Configuration Options:**
- **Model Selection**: Choose from various Amazon Bedrock models
- **Thinking Mode**: Enable step-by-step reasoning for complex analysis
- **Temperature Control**: Balance between consistency and creativity
- **Token Limits**: Control response length and computational cost

**Timeout and Retry Settings:**
- **Read Timeout**: 30 minutes for complex financial analysis
- **Connect Timeout**: 15 minutes for initial connection
- **Adaptive Retries**: Automatic retry with exponential backoff

In [3]:
import boto3
from botocore.config import Config
from strands.models import BedrockModel

# AWS Boto3 client configuration with timeouts and retries
boto_client_config = Config(
    read_timeout=1800,      # 30 minutes read timeout
    connect_timeout=900,    # 15 minutes connect timeout
    retries=dict(max_attempts=3, mode="adaptive"),
)


def get_model(model_id=NOVA_RPO_MODEL_ID, thinking=False,
              temperature=0.7, max_tokens=10000):
    """
    Create and return a Bedrock model instance.

    Args:
        model_id (str): The specific model ID to use
        thinking (bool): Whether to enable thinking mode for supported models
        temperature (float): Sampling temperature for response generation
        max_tokens (int): Maximum tokens in the response

    Returns:
        BedrockModel instance
    """
    session = boto3.Session()

    # Configure thinking mode for supported models
    additional_request_fields = {}
    if thinking:
        additional_request_fields = {
            "thinking": {
                "type": "enabled",
                "budget_tokens": 4096,
            }
        }

    return BedrockModel(
        model_id=model_id,
        boto_session=session,
        max_tokens=max_tokens,
        temperature=temperature,
        boto_client_config=boto_client_config,
        additional_request_fields=additional_request_fields,
    )

In [4]:
# Create a model instance that will be shared by all our agents
# This ensures consistent behavior across the multi-agent system
llm = get_model(NOVA_RPO_MODEL_ID)

## 3. Agent Definitions

### 3.1 Bear Researcher Agent

The Bear Researcher agent represents the cautious, risk-focused perspective in our investment analysis system. This agent is designed to:

**Core Responsibilities:**
- Identify and analyze potential investment risks and downsides
- Challenge overly optimistic assumptions with data-driven counterarguments
- Provide essential risk perspective to prevent overconfident investment decisions
- Engage in constructive debates with bullish analysts

**Analytical Framework:**
The Bear Researcher employs a comprehensive risk assessment framework covering:
- Financial and operational risks (debt levels, cash flow, margins)
- Market and competitive threats (saturation, disruption, regulation)
- Macroeconomic headwinds (recession, interest rates, inflation)
- Valuation concerns (overvaluation, market bubbles, volatility)

**Debate Strategy:**
- Present arguments with analytical rigor and healthy skepticism
- Use historical precedents and market cycles as supporting evidence
- Directly challenge bullish assumptions with concrete data
- Focus on downside protection and risk management

In [5]:
"""
Bear Researcher Agent
"""

from strands import Agent

def create_bear_researcher(llm, config):
    """
    Create a Bear Researcher agent that advocates for cautious investment positions.
    
    Args:
        llm: The language model to use for the agent
        config (dict): Configuration settings for the agent
        
    Returns:
        Agent: Configured Bear Researcher agent
    """
    
    system_message = (
        """You are a Bear Researcher specializing in risk analysis and investment caution. 
        Your role is to advocate against risky investments by presenting well-researched, 
        evidence-based arguments that emphasize potential downsides and market risks.

        **Key Responsibilities:**
        1. Build compelling cases against overvalued or risky investments
        2. Counter bullish arguments with solid risk analysis and data
        3. Engage in constructive debate with bull analysts
        4. Learn from past investment decisions and market downturns

        **Risk Assessment Framework:**

        **Financial and Operational Risks:**
        - Declining revenue trends and margin compression
        - High debt levels and liquidity concerns
        - Poor cash flow management and capital allocation
        - Overvaluation relative to fundamentals
        - Accounting irregularities or transparency issues

        **Market and Competitive Threats:**
        - Market saturation and limited growth opportunities
        - Intense competition and pricing pressure
        - Technological disruption and obsolescence risks
        - Loss of market share to competitors
        - Regulatory threats and compliance costs

        **Macroeconomic Headwinds:**
        - Economic recession or slowdown risks
        - Interest rate sensitivity and credit tightening
        - Inflation impact on costs and margins
        - Currency fluctuation risks for international exposure
        - Geopolitical tensions affecting operations

        **Industry-Specific Challenges:**
        - Cyclical downturns and seasonal volatility
        - Supply chain disruptions and cost inflation
        - Regulatory changes and policy shifts
        - Environmental and social governance risks
        - Technological shifts disrupting business models

        **Valuation and Market Concerns:**
        - Overvaluation relative to peers and historical metrics
        - Excessive market optimism and bubble indicators
        - Poor risk-adjusted returns and volatility
        - Liquidity concerns and market depth issues
        - Institutional selling pressure and insider activity

        **Debate Strategy:**
        - Present arguments in a conversational, analytical style
        - Directly challenge bull analyst assumptions with data
        - Highlight overlooked risks and potential downsides
        - Use historical precedents and market cycles as evidence
        - Expose weaknesses in bullish investment theses

        **Learning and Improvement:**
        - Use get_financial_situation_memories to review past market downturns
        - Identify patterns in market bubbles and corrections
        - Apply lessons from previous bear market calls
        - Refine risk assessment models based on historical accuracy

        **Communication Guidelines:**
        1. Present arguments with analytical rigor and skepticism
        2. Use concrete data and historical examples
        3. Challenge assumptions and highlight blind spots
        4. Maintain objectivity while being persuasively cautious
        5. Focus on downside protection and risk management
        6. Provide alternative scenarios and stress testing

        **Hand-off Guidelines:**
        - You should first give your own analysis.
        - If another researcher hasn't provided the final/completed analysis, you can directly hand off to the relevant researcher when you need to.

        Remember: Your goal is to provide essential risk perspective and protect against 
        overoptimistic investment decisions while maintaining analytical integrity.

        Need to Write your final analysis!
        """
    )

    # Create and configure the agent
    agent = Agent(
        model=llm,
        name="Bear Researcher",
        callback_handler=None,  # Disabled for parallel execution in debates
        system_prompt=system_message,
        load_tools_from_directory=False,
    )
    
    # Set agent state for configuration access
    agent.state.set("config", config)

    return agent

### 3.2 Bull Researcher Agent

The Bull Researcher agent represents the optimistic, growth-focused perspective in our investment analysis system. This agent is designed to:

**Core Responsibilities:**
- Identify and analyze investment opportunities and growth potential
- Build compelling cases for positive investment positions
- Counter pessimistic arguments with solid data and reasoning
- Highlight competitive advantages and market opportunities

**Analytical Framework:**
The Bull Researcher employs a comprehensive growth assessment framework covering:
- Growth potential (market expansion, revenue projections, innovation)
- Competitive advantages (differentiation, brand strength, market position)
- Financial strength (profitability, cash generation, balance sheet health)
- Market dynamics (trends, catalysts, positive sentiment)

**Debate Strategy:**
- Present arguments with data-driven optimism and evidence
- Use growth metrics and forward-looking indicators
- Counter bearish concerns with strategic advantages
- Focus on long-term value creation and market opportunities

In [6]:
"""
Bull Researcher Agent
"""

def create_bull_researcher(llm, config):
    """
    Create a Bull Researcher agent that advocates for positive investment positions.
    
    Args:
        llm: The language model to use for the agent
        config (dict): Configuration settings for the agent
        
    Returns:
        Agent: Configured Bull Researcher agent
    """
    
    system_message = (
        """You are a Bull Researcher specializing in building strong investment cases for stocks. 
        Your role is to advocate for positive investment positions by presenting well-researched, 
        evidence-based arguments that emphasize growth potential and competitive advantages.

        **Key Responsibilities:**
        1. Build compelling cases for investment opportunities
        2. Counter bearish arguments with solid data and reasoning
        3. Engage in constructive debate with bear analysts
        4. Learn from past investment decisions and mistakes

        **Analysis Framework:**

        **Growth Potential Assessment:**
        - Market opportunity size and expansion potential
        - Revenue growth projections and scalability factors
        - New product launches and innovation pipeline
        - Geographic expansion opportunities
        - Market share growth potential

        **Competitive Advantages:**
        - Unique products or services with strong differentiation
        - Brand strength and customer loyalty
        - Dominant market positioning and barriers to entry
        - Technological advantages and intellectual property
        - Cost advantages and operational efficiency

        **Positive Financial Indicators:**
        - Strong financial health metrics (revenue, profit margins, cash flow)
        - Improving financial trends and performance metrics
        - Strong balance sheet and low debt levels
        - Efficient capital allocation and return on investment
        - Dividend growth and shareholder returns

        **Market and Industry Trends:**
        - Favorable industry tailwinds and secular trends
        - Positive regulatory environment changes
        - Increasing demand for company's products/services
        - Supply chain advantages and partnerships
        - ESG (Environmental, Social, Governance) strengths

        **Debate Strategy:**
        - Present arguments in a conversational, engaging style
        - Directly address and counter bear analyst concerns
        - Use specific data and evidence to support positions
        - Acknowledge risks while demonstrating why positives outweigh negatives
        - Build on previous arguments and strengthen the investment thesis

        **Learning and Improvement:**
        - Use get_financial_situation_memories to review past investment decisions
        - Identify patterns in successful and unsuccessful investment theses
        - Apply lessons learned to current analysis and recommendations
        - Continuously refine analytical approach based on historical performance

        **Communication Guidelines:**
        1. Present arguments conversationally and engagingly
        2. Use specific data points and concrete evidence
        3. Address counterarguments proactively and thoroughly
        4. Build momentum in debates by reinforcing strong points
        5. Maintain professional tone while being persuasive
        6. Focus on actionable insights and clear investment rationale

        **Hand-off Guidelines:**
        - You should first give your own analysis, and then directly hand off to the relevant researcher when you need to.
        - If another researcher hasn't provided the final/completed analysis, you can directly hand off to the relevant researcher when needed. Otherwise, you cannot hand off to them.


        Remember: Your goal is to build the strongest possible case for investment while 
        maintaining intellectual honesty and acknowledging legitimate concerns.

        Need to Write your final analysis!
        """
    )

    # Create and configure the agent
    agent = Agent(
        model=llm,
        name="Bull Researcher",
        callback_handler=None,  # Disabled for parallel execution in debates
        system_prompt=system_message,
        load_tools_from_directory=False,
    )
    
    # Set agent state for configuration access
    agent.state.set("config", config)

    return agent

### 3.3 Research Manager Agent

The Research Manager agent serves as the senior decision-maker who synthesizes insights from both Bull and Bear researchers to make final investment recommendations. This agent represents the role of a portfolio manager or investment committee chair.

**Core Responsibilities:**
- Coordinate and facilitate debates between bull and bear analysts
- Critically evaluate arguments from multiple perspectives
- Synthesize conflicting viewpoints into coherent investment decisions
- Provide clear, actionable investment recommendations with rationale

**Decision-Making Framework:**
- **Argument Evaluation**: Assess the strength and validity of bull vs bear cases
- **Risk-Return Analysis**: Balance potential upside against downside risks
- **Implementation Strategy**: Define specific actions (Buy/Sell/Hold) with timing
- **Risk Management**: Establish position sizing and stop-loss parameters

**Synthesis Approach:**
- Identify the most compelling evidence from both sides
- Resolve contradictions through deeper analysis
- Make decisions based on probability-weighted outcomes
- Communicate decisions with clear reasoning and confidence levels

In [7]:
"""
Research Manager Agent
"""

def create_research_manager(llm, config):
    """
    Create a Research Manager agent that coordinates debates and makes investment decisions.
    
    Args:
        llm: The language model to use for the agent
        config (dict): Configuration settings for the agent
        
    Returns:
        Agent: Configured Research Manager agent
    """
    
    system_message = (
        """You are a Research Manager and Portfolio Manager responsible for coordinating 
        research team debates and making definitive investment decisions. Your role is to 
        critically evaluate arguments from both bull and bear analysts and provide clear, 
        actionable investment recommendations.

        **Key Responsibilities:**
        1. Facilitate and evaluate research team debates
        2. Make definitive investment decisions (Buy, Sell, or Hold)
        3. Develop detailed investment plans for the trading team
        4. Learn from past investment decisions and outcomes

        **Decision-Making Framework:**

        **Debate Evaluation Process:**
        - Critically assess the strength of bull and bear arguments
        - Identify the most compelling evidence and reasoning from each side
        - Evaluate the quality of data and analysis presented
        - Consider risk-reward ratios and probability-weighted outcomes
        - Assess market timing and execution feasibility

        **Investment Decision Criteria:**
        - **Buy**: Strong bull case with manageable risks and favorable risk-reward
        - **Sell**: Significant risks outweigh potential returns or overvaluation concerns
        - **Hold**: Only when genuinely balanced arguments justify maintaining position
        - Avoid defaulting to Hold - commit to a clear stance based on evidence

        **Investment Plan Development:**
        Create comprehensive plans including:
        - Clear recommendation with supporting rationale
        - Risk assessment and mitigation strategies
        - Position sizing and entry/exit criteria
        - Timeline and milestone monitoring
        - Contingency plans for different scenarios

        **Risk Management Considerations:**
        - Portfolio diversification and concentration limits
        - Correlation with existing positions
        - Liquidity requirements and market conditions
        - Volatility expectations and drawdown limits
        - Regulatory and compliance considerations

        **Learning and Improvement:**
        - Review past investment decisions and their outcomes
        - Identify patterns in successful and unsuccessful calls
        - Analyze debate quality and decision-making processes
        - Refine evaluation criteria based on historical performance
        - Update investment frameworks based on market evolution

        **Communication Style:**
        1. Present analysis conversationally and naturally
        2. Avoid excessive formatting or bullet points
        3. Provide clear, decisive recommendations
        4. Explain reasoning in accessible terms
        5. Address both opportunities and risks transparently
        6. Focus on actionable insights for the trading team

        **Decision Output Requirements:**
        - **Investment Recommendation**: Clear Buy/Sell/Hold decision
        - **Rationale**: Explanation of why this decision is optimal
        - **Strategic Actions**: Concrete implementation steps
        - **Risk Factors**: Key risks and mitigation approaches
        - **Success Metrics**: How to measure investment performance
        - **Review Timeline**: When to reassess the position

        **Tools Available:**
        - get_financial_situation_memories: Review past investment decisions and lessons
        - Note: Do NOT use add_financial_situation_memories during evaluation

        **Critical Guidelines:**
        - Make decisive recommendations based on the strongest arguments
        - Don't default to Hold unless truly justified by balanced evidence
        - Consider both short-term and long-term implications
        - Account for market conditions and timing factors
        - Maintain objectivity while being actionable
        - Learn from past mistakes to improve future decisions

        Remember: Your role is to synthesize complex debates into clear, actionable 
        investment strategies that maximize risk-adjusted returns for the portfolio.
        """
    )

    # Create and configure the agent
    agent = Agent(
        model=llm,
        name="Research Manager",
        system_prompt=system_message,
        load_tools_from_directory=False,
    )
    
    # Set agent state for configuration access
    agent.state.set("config", config)
    
    return agent

## 4. Multi-Agent Coordination System

### 4.1 Conversation Swarm Implementation

The ConversationSwarm class orchestrates sophisticated multi-agent debates and discussions. This system enables our Bull and Bear researchers to engage in structured, productive conversations that lead to well-reasoned investment decisions.

**Key Features:**
- **Structured Debates**: Manages turn-taking and ensures balanced participation
- **Dynamic Handoffs**: Agents can transfer control based on expertise areas
- **Timeout Management**: Prevents infinite loops and ensures timely decisions
- **Conversation History**: Tracks all interactions for analysis and learning

**Execution Phases:**
1. **Initial Analysis**: Both agents provide independent perspectives
2. **Interactive Debate**: Agents challenge and refine each other's arguments
3. **Synthesis**: Research Manager creates final investment recommendation

**Configuration Parameters:**
- **Max Handoffs**: Limits conversation length (prevents endless debates)
- **Execution Timeout**: Overall time limit for the entire conversation
- **Node Timeout**: Time limit for individual agent responses
- **Repetition Detection**: Prevents circular arguments and ensures progress

In [8]:
"""
Conversation Swarm
"""

import logging
from strands.multiagent import Swarm


# Enable debug logs and print them to stderr
logging.getLogger("strands.multiagent").setLevel(logging.DEBUG)
logging.basicConfig(
    format="%(levelname)s | %(name)s | %(message)s",
    handlers=[logging.StreamHandler()]
)


class ConversationSwarm:
    """
    A multi-agent conversation system for coordinating debates and discussions.
    
    This class manages interactions between multiple agents, allowing them to
    collaborate, compete, or use hybrid approaches to analyze trading scenarios
    and reach consensus decisions.
    """
    
    def __init__(self, agents, summarizer_agent):
        """
        Initialize the ConversationSwarm with agents and collabrative strategy.
        
        Args:
            agents (list): List of agent instances to participate in discussions
            summarizer_agent: Agent responsible for synthesizing final decisions
        """
        self.agents = agents
        self.summarizer_agent = summarizer_agent
        
    def run(self, task):
        """
        Execute a multi-phase conversation between agents to analyze a task.
        
        The conversation follows a structured approach:
        1. Initial parallel analysis by all agents
        2. Refinement phase where agents respond to each other's insights
        3. Final synthesis by the summarizer agent
        
        Args:
            task (str): The trading task or question to analyze
            
        Returns:
            tuple: (final_solution, messages_dict) containing the synthesized
                   decision and all conversation messages
        """
        
        # Initialize message dictionary to track all agent communications
        messages = {}
        messages[self.summarizer_agent.name] = []
        for agent in self.agents:
            messages[agent.name] = []
        
        print(f"Starting collabrative conversation with {len(self.agents)} agents...")
        # Create a swarm with these agents
        swarm = Swarm(
            self.agents,
            max_handoffs=10,
            max_iterations=10,
            execution_timeout=2400.0,  # 15 minutes
            node_timeout=600.0,       # 5 minutes per agent
            repetitive_handoff_detection_window=8,  # There must be >= 3 unique agents in the last 8 handoffs
            repetitive_handoff_min_unique_agents=5
        )

        # Execute the swarm on a task
        print("Phase 1: Swarm conversation between bull reseacher and bear reseacher...")
        result = swarm(task)

        # Access the final result
        print(f"Status: {result.status}")
        print(f"Node history: {[node.node_id for node in result.node_history]}")
        save_as_file(",".join([node.node_id for node in result.node_history]), working_dir, prefix, "node_history.txt")
        
        bull_history, bear_history = None, None
        for message in self.agents[0].messages[::-1]:
            if message["role"] != "assistant" or len(message["content"]) == 0:
                continue
            else:
                for item in message["content"]:
                    if "text" in item:
                        bull_history = item["text"]
                break

        for message in self.agents[1].messages[::-1]:
            if message["role"] != "assistant" or len(message["content"]) == 0:
                continue
            else:
                for item in message["content"]:
                    if "text" in item:
                        bear_history = item["text"]
                break
        print("Phase 2: Final synthesis and decision...")
        
        summarizer_prompt = f"""
Original Investment Analysis Task:
<query>
{task}
</query>

You have received comprehensive analyses from the research team. Please synthesize 
these inputs into a final investment decision and strategy:

<team_analyses>
Bull Reseacher: 
{bull_history}

Bear Reseacher:
{bear_history}
</team_analyses>

Your synthesis should:
1. Evaluate the strength of arguments from both bull and bear perspectives
2. Identify the most compelling evidence and reasoning
3. Make a clear investment recommendation (Buy, Sell, or Hold)
4. Provide a detailed rationale for your decision
5. Outline specific implementation strategies
6. Address key risks and mitigation approaches

Create a comprehensive final investment plan that incorporates the best insights 
from the team while addressing any concerns or contradictions in their analyses.
"""

        print("Generating final synthesis...")
        final_solution = self.summarizer_agent(summarizer_prompt)
        
        print("Conversation completed successfully!")
        
        return final_solution, bull_history, bear_history

## 5. System Execution and Analysis

### 5.1 Initialize Multi-Agent Investment Analysis

This section demonstrates how to set up and execute our multi-agent investment analysis system. We'll create instances of our agents and coordinate their collaboration through the Swarm framework.

**Setup Process:**
1. **Configuration**: Define analysis parameters (stock ticker, date, output directory)
2. **Agent Creation**: Instantiate Bull Researcher, Bear Researcher, and Research Manager
3. **Swarm Assembly**: Create ConversationSwarm with proper coordination settings
4. **Data Loading**: Import relevant market reports and news for context
5. **Execution**: Run the multi-agent analysis and capture results

**Key Configuration Variables:**
- `company_of_interest`: Stock ticker symbol for analysis (e.g., 'AMZN')
- `trade_date`: Analysis date for temporal context
- `working_dir`: Output directory for saving analysis results
- `prefix`: File naming convention for organized output

In [9]:
from model_utils import get_model, save_as_file, read_file
from default_config import DEFAULT_CONFIG
# Configuration
company_of_interest = "AMZN"  # Stock ticker to analyze
trade_date = "2025-08-19"    # Analysis date
working_dir = DEFAULT_CONFIG['results_dir']
prefix = f"{company_of_interest}_{trade_date}".replace(" ", "_")
config = DEFAULT_CONFIG.copy()

bull_researcher = create_bull_researcher(llm, config)
bear_researcher = create_bear_researcher(llm, config)
research_manager = create_research_manager(llm, config)

# Create debate swarm with collabrative coordination
research_debate = ConversationSwarm(
    agents=[bull_researcher, bear_researcher],
    summarizer_agent=research_manager
)

# Load previous reports for context
market_report = read_file(working_dir, prefix, "market_report.txt")
news_report = read_file(working_dir, prefix, "news_report.txt")

### 5.2 Execute Multi-Agent Investment Analysis

This section demonstrates the execution of our multi-agent investment analysis system. The process involves:

**Execution Flow:**
1. **Agent Initialization**: Create Bull Researcher, Bear Researcher, and Research Manager instances
2. **Context Loading**: Import market reports and news data for informed analysis
3. **Swarm Execution**: Run structured debate between Bull and Bear researchers
4. **Synthesis**: Research Manager creates final investment recommendation
5. **Output Generation**: Save all analyses and decisions to files

**Debug Information:**
The debug logs show the detailed execution flow:
- Node handoffs between Bull and Bear researchers
- Iteration counts and timing information
- Completion status and execution metrics

**Expected Outcomes:**
- Comprehensive bull and bear analyses
- Structured debate with multiple rounds of interaction
- Final synthesized investment recommendation
- Detailed execution logs for analysis and improvement

In [10]:
# Run the debate
print("Step 3: Running research team debate...")
investment_plan, bull_history, bear_history = research_debate.run(
    f"collaborative multi-agent analysis and decide on an investment plan for {company_of_interest} "
    f"for the trade date {trade_date} based on the following reports:\n\n"
    f"Market Report:\n{market_report}\n\n"
    f"News Report:\n{news_report}"
)

# save final analysis
save_as_file(bull_history, working_dir, prefix, "bull_final_analysis.txt")
save_as_file(bear_history, working_dir, prefix, "bear_final_analysis.txt")
# Save investment plan
save_as_file(str(investment_plan), working_dir, prefix, "investment_plan.txt")
print("Research team debate completed.")

DEBUG | strands.multiagent.swarm | nodes=<['Bull Researcher', 'Bear Researcher']> | initialized swarm with nodes
DEBUG | strands.multiagent.swarm | tool_count=<1>, node_count=<2> | injected coordination tools into agents
DEBUG | strands.multiagent.swarm | starting swarm execution
DEBUG | strands.multiagent.swarm | current_node=<Bull Researcher> | starting swarm execution with node
DEBUG | strands.multiagent.swarm | max_handoffs=<10>, max_iterations=<10>, timeout=<2400.0>s | swarm execution config
DEBUG | strands.multiagent.swarm | current_node=<Bull Researcher>, iteration=<1> | executing node


Step 3: Running research team debate...
Starting collabrative conversation with 2 agents...
Phase 1: Swarm conversation between bull reseacher and bear reseacher...


DEBUG | strands.multiagent.swarm | from_node=<Bull Researcher>, to_node=<Bear Researcher> | handed off from agent to agent
DEBUG | strands.multiagent.swarm | node=<Bull Researcher> | node execution completed
DEBUG | strands.multiagent.swarm | current_node=<Bear Researcher>, iteration=<2> | executing node
DEBUG | strands.multiagent.swarm | from_node=<Bear Researcher>, to_node=<Bull Researcher> | handed off from agent to agent
DEBUG | strands.multiagent.swarm | node=<Bear Researcher> | node execution completed
DEBUG | strands.multiagent.swarm | current_node=<Bull Researcher>, iteration=<3> | executing node
DEBUG | strands.multiagent.swarm | from_node=<Bull Researcher>, to_node=<Bear Researcher> | handed off from agent to agent
DEBUG | strands.multiagent.swarm | node=<Bull Researcher> | node execution completed
DEBUG | strands.multiagent.swarm | current_node=<Bear Researcher>, iteration=<4> | executing node
DEBUG | strands.multiagent.swarm | from_node=<Bear Researcher>, to_node=<Bull Rese

Status: Status.COMPLETED
Node history: ['Bull Researcher', 'Bear Researcher', 'Bull Researcher', 'Bear Researcher', 'Bull Researcher', 'Bear Researcher', 'Bull Researcher', 'Bear Researcher']
Phase 2: Final synthesis and decision...
Generating final synthesis...
### Final Investment Decision and Strategy for Amazon (AMZN) on 2025-08-19

#### **Evaluation of Bull and Bear Arguments**

**Bull Perspective:**
- **Strong Growth Drivers:** AWS, advertising, and capex continue to drive long-term growth.
- **Positive Momentum:** Technical indicators like MACD and VWMA suggest positive momentum and strong buying volume.
- **Upside Potential:** Analysts predict a median price target of $264.21, indicating significant upside potential.
- **Bullish Sentiment:** Major tech stocks, including AMZN, are forecasted to be bullish.

**Bear Perspective:**
- **Slower Growth:** Concerns over Amazon's slower revenue growth compared to peers.
- **High Debt and Liquidity Concerns:** High capex leading to poten

## 3 Complete Task 2

Find the analysis just generated, the directory is located `./results/`.

In [11]:
stock_data_dir='./results/'

In [12]:
# Display the conversation flow between agents
!cat {stock_data_dir}/{prefix}/node_history.txt

Bull Researcher,Bear Researcher,Bull Researcher,Bear Researcher,Bull Researcher,Bear Researcher,Bull Researcher,Bear Researcher

## 6. Analysis Results and Insights

### 6.1 Multi-Agent Conversation Flow

The node history above shows the structured conversation flow between our Bull and Bear researchers. This demonstrates:

**Conversation Pattern:**
- **Alternating Perspectives**: Bull and Bear researchers take turns presenting arguments
- **Dynamic Handoffs**: Agents transfer control based on the conversation context
- **Structured Debate**: Each agent builds upon and challenges the other's analysis

**Key Observations:**
- The conversation involved 8 total interactions between the agents
- Both agents had equal opportunity to present their perspectives
- The debate concluded naturally when both sides had thoroughly explored their positions

### 6.2 Individual Agent Analyses

Below are the final analyses from each researcher, showcasing their distinct analytical approaches and conclusions.

### 6.3 Bear Researcher Analysis

The Bear Researcher focuses on risk identification and cautionary perspectives. Key analytical themes include:
- **Valuation Concerns**: Analysis of overvaluation relative to peers and historical metrics
- **Market Sentiment Risks**: Assessment of excessive optimism and bubble indicators
- **Risk-Adjusted Returns**: Evaluation of volatility and Sharpe ratios
- **Liquidity and Institutional Factors**: Analysis of trading patterns and institutional behavior

In [13]:
# Display the Bear Researcher's final analysis
!cat {stock_data_dir}/{prefix}/bear_final_analysis.txt

<thinking> It appears that the specialized agents required for the detailed analysis are not available in the current swarm. Therefore, I will need to consolidate the information myself and provide a comprehensive risk assessment based on the available data and my own analysis. 

I will summarize the key points from the previous analyses and integrate the additional areas requested in the handoff message as best as I can with the information at hand. </thinking>

Based on the available data and previous analyses, here is a comprehensive risk assessment for investing in Amazon (AMZN) on the trade date 2025-08-19:

### Financial and Operational Risks
- **Declining Revenue Trends and Margin Compression:** Although Amazon reported a 13% YoY revenue growth in Q2, concerns over slower growth compared to peers suggest potential margin compression in the future.
- **High Debt Levels and Liquidity Concerns:** Amazon's high levels of capital expenditure (capex) could lead to increased debt level

In [14]:
# Display the Bull Researcher's final analysis
!cat {stock_data_dir}/{prefix}/bull_final_analysis.txt

<thinking> Now that I have handed off the request to the Bear Researcher for additional analysis, I will wait for their response to incorporate into the final investment plan for AMZN. </thinking>

**Status Update:**
I have handed off the request to the Bear Researcher for additional analysis on the specified areas to enhance the risk assessment for investing in Amazon (AMZN) on the trade date 2025-08-19. Once I receive their analysis, I will incorporate it into the final investment plan.

**Next Steps:**
- Await the response from the Bear Researcher.
- Incorporate the additional analysis into the final investment plan.
- Provide a comprehensive recommendation based on the combined analysis.

### 6.4 Bull Researcher Analysis

The Bull Researcher focuses on growth opportunities and positive investment factors. Key analytical themes include:
- **Growth Potential**: Analysis of market opportunities and expansion prospects
- **Competitive Advantages**: Assessment of unique strengths and market positioning
- **Financial Performance**: Evaluation of profitability trends and financial health
- **Long-term Value Creation**: Focus on sustainable competitive advantages and innovation

## 🎉 Congratulations!

You have successfully completed **Challenge 2** and built a sophisticated multi-agent investment analysis system!

### What You've Mastered:
- ✅ **Multi-Agent System Design**: Created specialized agents with distinct roles and responsibilities
- ✅ **Agent Coordination**: Implemented structured debates using the Strands Swarm framework
- ✅ **Financial Analysis AI**: Built agents capable of sophisticated investment analysis and debate
- ✅ **Collaborative Intelligence**: Demonstrated how AI agents can work together to reach better decisions

### Ready for the Next Challenge?
You're now prepared to continue to **Challenge 3**, where you'll explore even more advanced agent capabilities and coordination patterns.

Great work on mastering the fundamentals of multi-agent AI systems! 🚀